In [2]:
import pandas as pd
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, TextGenerationPipeline
import matplotlib.pyplot as plt
from dotenv import load_dotenv 
load_dotenv()

/home/zidane/anaconda3/envs/proj_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
device = "cuda"

In [3]:
# path = "/home/zidane/.cache/huggingface/hub/models--mistralai--Mistral-7B-Instruct-v0.2/snapshots/41b61a33a2483885c981aa79e0df6b32407ed873"
# tokenizer = AutoTokenizer.from_pretrained(path)
# model = AutoModelForCausalLM.from_pretrained(path)
# device = "cuda"

In [3]:
deepseek_model = {
    "tokenizer": AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct-v1.5"),
    "model" : AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-7b-instruct-v1.5")
}

# mistral_model = {
#     "tokenizer": AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2"),
#     "model" : AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
# }

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.29s/it]


In [5]:
!huggingface-cli scan-cache

REPO ID                                                     REPO TYPE SIZE ON DISK NB FILES LAST_ACCESSED  LAST_MODIFIED REFS LOCAL PATH                                                                                      
----------------------------------------------------------- --------- ------------ -------- -------------- ------------- ---- ----------------------------------------------------------------------------------------------- 
CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment           model           436.7M        5 3 weeks ago    3 weeks ago   main /disk/data/huggingface/hub/models--CAMeL-Lab--bert-base-arabic-camelbert-da-sentiment           
MohamedGalal/marbert-sarcasm-detector                       model           655.2M        6 3 weeks ago    3 weeks ago   main /disk/data/huggingface/hub/models--MohamedGalal--marbert-sarcasm-detector                       
deepseek-ai/deepseek-coder-33b-instruct                     model             9.7G        5 2 days ago     2

In [3]:
df = pd.read_csv('titanic.csv')

In [7]:
str(list(df.columns))

"['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']"

In [6]:
print(df.loc[0])

Survived                                        0
Pclass                                          3
Name                       Mr. Owen Harris Braund
Sex                                          male
Age                                          22.0
Siblings/Spouses Aboard                         1
Parents/Children Aboard                         0
Fare                                         7.25
Name: 0, dtype: object


In [48]:
import re
def extract_response(text):
    pattern = r"```python(.*?)```"
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        return {'response_error':False,'response':matches[0]}
    else:
        return {'response_error':True,'response':text}

def generate_response(prompt,model_name):
    try:
        coder_model_prompt = [
        {"role": "user", "content": prompt}
        ]
        encodeds = model_name["tokenizer"].apply_chat_template(coder_model_prompt, return_tensors="pt")

        model_inputs = encodeds.to(device)
        model_name['model'].to(device)

        generated_ids = model_name['model'].generate(model_inputs, max_new_tokens=500, do_sample=False,temperature=0.1,repetition_penalty=1)
        decoded = model_name["tokenizer"].batch_decode(generated_ids)
        return decoded[0].split('[/INST]')[-1].split('</s>')[0]
    except:
        return

In [55]:
prompt_template = """df is a dataframe that {description}. df has these columns: {columns}. Write python code that answers this question: Print {question}
"""

In [33]:
titanic_description = 'containing the list of people that were on the titanic ship and some details such as age, sex, name and whether they survived or not'
titanic_columns = '''['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']'''

In [13]:
# 1. What is the fare for Mr. Owen Harris Braund?
answer1 = df[df['Name'] == "Mr. Owen Harris Braund"]['Fare'].iloc[0]

# 2. How many females are in the dataset?
answer2 = df[df['Sex'] == "female"].shape[0]

# 3. What is the average age of the passengers?
answer3 = df['Age'].mean()

# 4. How many passengers survived?
answer4 = df[df['Survived'] == 1].shape[0]

# 5. Who paid the highest fare?
answer5 = df.loc[df['Fare'].idxmax(), 'Name']

# 6. What is the total amount of fare paid?
answer6 = df['Fare'].sum()

# 7. Who is the passenger that has the highest number of siblings abroad?
answer7 = df.loc[df['Siblings/Spouses Aboard'].idxmax(), 'Name']

# 8. What is the age of Miss. Laina Heikkinen?
answer8 = df[df['Name'] == "Miss. Laina Heikkinen"]['Age'].iloc[0]

# 9. How many passengers are in the 1st class?
answer9 = df[df['Pclass'] == 1].shape[0]

# 10. What is the average fare paid by passengers?
answer10 = df['Fare'].mean()

# 11. How many passengers have 0 siblings/spouses aboard?
answer11 = df[df['Siblings/Spouses Aboard'] == 0].shape[0]

# 12. Who is the oldest passenger in the dataset?
answer12 = df.loc[df['Age'].idxmax(), 'Name']

# 13. How many passengers did not survive?
answer13 = df[df['Survived'] == 0].shape[0]

# 14. What is the lowest fare paid in the dataset?
answer14 = df['Fare'].min()

test_cases_titanic = [
    {'question': 'What is the fare for Mr. Owen Harris Braund?', 'answer': answer1},
    {'question': 'How many females are in the dataset?', 'answer': answer2},
    {'question': 'What is the average age of the passengers?', 'answer': answer3},
    {'question': 'How many passengers survived?', 'answer': answer4},
    {'question': 'Who paid the highest fare?', 'answer': answer5},
    {'question': 'What is the total amount of fare paid?', 'answer': answer6},
    {'question': 'Who is the passanger that has the highest number of siblings abroad ?', 'answer': answer7},
    {"question": "What is the age of Miss. Laina Heikkinen?", "answer": answer8},
    {"question": "How many passengers are in the 1st class?", "answer": answer9},
    {"question": "What is the average fare paid by passengers?", "answer": answer10},
    {"question": "How many passengers have 0 siblings/spouses aboard?", "answer": answer11},
    {"question": "Who is the oldest passenger in the dataset?", "answer": answer12},
    {"question": "How many passengers did not survive?", "answer": answer13},
    {"question": "What is the lowest fare paid in the dataset?", "answer": answer14}
]

In [43]:
# 1. What is the monthly income of the first female in the dataset?
answer1 = df.loc[df['Gender'] == "Female", 'Monthly Income'].iloc[0]

# 2. How many students are there in the dataset?
answer2 = (df['Occupation'] == "Student").sum()

# 3. What is the average age of participants?
answer3 = df['Age'].mean()

# 4. How many participants provided positive feedback?
answer4 = (df['Feedback'] == "Positive").sum()

# 5. What is the educational qualification of the male participant?
answer5 = df.loc[df['Gender'] == "Male", 'Educational Qualifications'].iloc[0]

# 6. How many participants are single?
answer6 = (df['Marital Status'] == "Single").sum()

# 7. What is the most common marital status in the dataset?
answer7 = df['Marital Status'].mode()[0]

# 8. Are there any participants with a family size greater than 3?
answer8 = "Yes" if any(df['Family size'] > 3) else "No"

# 9. What is the pin code for the location with the highest latitude?
answer9 = df.loc[df['latitude'].idxmax(), 'Pin code']

# 10. Did any participant with negative feedback have a monthly income below Rs.10000?
answer10 = "Yes" if any((df['Feedback'] == "Negative") & (df['Monthly Income'] == "Below Rs.10000")) else "No"

test_cases_food=[
    {"question": "What is the monthly income of the first female in the dataset?", "answer": answer1},
    {"question": "How many students are there in the dataset?", "answer": answer2},
    {"question": "What is the average age of participants?", "answer": answer3},
    {"question": "How many participants provided positive feedback?", "answer": answer4},
    {"question": "What is the educational qualification of the male participant?", "answer": answer5},
    {"question": "How many participants are single?", "answer": answer6},
    {"question": "What is the most common marital status in the dataset?", "answer": answer7},
    {"question": "Are there any participants with a family size greater than 3?", "answer": answer8},
    {"question": "What is the pin code for the location with the highest latitude?", "answer": answer9},
    {"question": "Did any participant with negative feedback have a monthly income below Rs.10000?", "answer": answer10}
]


In [50]:
import io
import sys
def get_print(code):
    buffer = io.StringIO()
    sys.stdout = buffer

    exec(code)

    sys.stdout = sys.__stdout__

    return buffer.getvalue()

In [52]:
food_description = 'The dataset contains information collected from an online food ordering platform over a period of time.'
food_columns = "['Age', 'Gender', 'Marital Status', 'Occupation', 'Monthly Income', 'Educational Qualifications', 'Family size', 'latitude', 'longitude', 'Pin code', 'Output', 'Feedback', 'Unnamed: 12']"

In [ ]:
data = []  
from tqdm import tqdm
for tc in tqdm(test_cases_food):
    prompt = prompt_template.format(description=food_description,columns=food_columns,question=tc['question'])
    full_response = generate_response(prompt,deepseek_model)
    extracted_code = extract_response(full_response)
    try:
        execution = get_print(extracted_code['response'])
    except:
        execution = 'Execution error'
    data.append({
        'question': tc['question'],
        'answer': tc['answer'],
        'response_error': extracted_code.get('response_error', None),  # Use .get to avoid KeyError if 'error' is missing
        'response': extracted_code.get('response', None),
        'execution': execution
    })

# Once the loop is complete, convert the list of dictionaries into a DataFrame
testdf = pd.DataFrame(data)
testdf.to_excel('food_results.xlsx')

In [3]:
df.head()

,Index,"Height(Inches)""","""Weight(Pounds)"""
0,1,65.78,112.99
1,2,71.52,136.49
2,3,69.40,153.03
3,4,68.22,142.34
4,5,67.79,144.30


In [6]:
df.columns

Index(['Index', ' Height(Inches)"', ' "Weight(Pounds)"'], dtype='object')

In [9]:
import pandas as pd
df = pd.read_csv('hw_200.csv')
# Assuming df is your DataFrame
highest_person_index = df['Index'][df[' Height(Inches)"'] == df[' Height(Inches)"'].max()]
print(highest_person_index)

138    139
Name: Index, dtype: int64
